In [19]:
!pip install opendatasets

In [20]:
import opendatasets as od
import numpy as np
import cv2
import glob
import gc

from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Input
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D

In [21]:
gc.collect()

3846

In [22]:
od.download(
    "https://www.kaggle.com/datasets/pranavraikokte/covid19-image-dataset"
)

Skipping, found downloaded files in "./covid19-image-dataset" (use force=True to force download)


In [23]:
labels=[
    "Covid",
    "Normal",
    "Viral Pneumonia"
]

In [24]:
train_images=[]
train_labels=[]

for a,i in enumerate(labels):
  for train_image in glob.iglob("/kaggle/working/covid19-image-dataset/Covid19-dataset/train/%s/*"% i):
    im = cv2.imread(train_image, cv2.IMREAD_GRAYSCALE)
    im = im.astype(np.float64) / 255
    im = cv2.resize(im,(32,32))
    train_images.append(np.array(im))
    train_labels.append(a)
len(train_images)

251

In [25]:
test_images=[]
test_labels=[]

for a,i in enumerate(labels):
  for test_image in glob.iglob("/kaggle/working/covid19-image-dataset/Covid19-dataset/test/%s/*"% i):
    im = cv2.imread(test_image, cv2.IMREAD_GRAYSCALE)
    im = im.astype(np.float64) / 255
    im = cv2.resize(im,(32,32))
    test_images.append(np.array(im))
    test_labels.append(a)
len(test_images)

66

In [26]:
lenet5_model=Sequential([
   Conv2D(6, (5, 5), activation='tanh', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (5, 5), activation='tanh'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(120, activation='tanh'),
    Dense(84, activation='tanh'),
    Dense(len(labels), activation='softmax')
])

In [27]:
lenet5_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 6)         0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 400)               0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [28]:
lenet5_model.compile(Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
lenet5_model.fit(
        np.array(train_images),
        np.array(train_labels),
        batch_size=1,
        epochs=20,
        validation_split=0.2,
)

Epoch 1/20
200/200 [==============================] - 3s 4ms/step - loss: 0.7414 - accuracy: 0.7250 - val_loss: 1.1957 - val_accuracy: 0.2157
Epoch 2/20
200/200 [==============================] - 1s 4ms/step - loss: 0.3022 - accuracy: 0.8900 - val_loss: 1.2257 - val_accuracy: 0.2549
Epoch 3/20
200/200 [==============================] - 1s 3ms/step - loss: 0.2367 - accuracy: 0.8850 - val_loss: 0.1888 - val_accuracy: 0.9412
Epoch 4/20
200/200 [==============================] - 1s 3ms/step - loss: 0.1739 - accuracy: 0.9400 - val_loss: 1.0921 - val_accuracy: 0.5294
Epoch 5/20
200/200 [==============================] - 1s 3ms/step - loss: 0.1301 - accuracy: 0.9500 - val_loss: 0.7944 - val_accuracy: 0.6863
Epoch 6/20
200/200 [==============================] - 1s 3ms/step - loss: 0.0911 - accuracy: 0.9750 - val_loss: 0.5894 - val_accuracy: 0.7843
Epoch 7/20
200/200 [==============================] - 1s 3ms/step - loss: 0.0255 - accuracy: 0.9950 - val_loss: 0.6585 - val_accuracy: 0.7059
Epoch 

In [30]:
lenet5_model.evaluate(
        np.array(test_images),
        np.array(test_labels),
        batch_size=1,
)

66/66 [==============================] - 0s 2ms/step - loss: 0.2177 - accuracy: 0.9394


[0.2176627516746521, 0.939393937587738]